
---
<big><big><big><big><big><big>Sieci neuronowe 2018/19</big></big></big></big></big></big>

---
<big><big><big><big><big>Adwersarialny autoenkoder</big></big></big></big></big>

---

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("fivethirtyeight")

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure, output_notebook
from bkcharts import Scatter

In [ ]:
output_notebook()

In [ ]:
sns.set(font_scale=2.0)

Image inclusion
<img src="../nn_figures/" width="100%">

# Quiz

Dzisiaj, czwartego czerwca, mamy rocznice.
* Czego?
* Które?

# Adwersarialny autoenkoder
* auto-enkodery mają przewagę nad tradycyjną architekturą GANów w tym, że zawierają enkoder
* uczenie adwersarialne daje ostrzejsze obrazy
  * ma też swoje wady…
  
<img src="../nn_figures/adversarial_autoencoder.pdf" width="90%">  
* adwersarialne uczenie
  * przetworzone przez enkoder przykłady $z\sim q(z)$ jako przykłady _fałszywe_ (negative, fake) $$q(z)=\int_x q(z\mid x)p_X(x)dx,$$
  gdzie $p_X(x)$ jest prawdziwym rozkładem danych, a $q(z)$ jest rozkładem kodowania
  * adwersarialny autoenkoder __dopasowuje__ posterior $q(z)$ do założonego prioru $p(z)$
  * przykłady $z\sim p(z)$ są losowane z prioru
  * adwersarialny koszt dyskryminatora jest kosztem rozpoznawanie pozytywnych $p(z)$ od negatywnych $q(z)$
  * można powiedzieć, że _generatorem_ modelu jest autoenkoder $q(z\mid x)$
  * dekoder odtwarza $z$
  

## Uczenie AAE
* równoległe uczenie sieci adwersarialnej (dyskryminatora) i autoenkodera
  * faza __rekonstrukcja__ 
    * aktualizuje enkoder i dekoder dla minimalizacji błędu
  * faza __regularyzacji__ w której sieć adwersarialna
    1. poprawia dyskryminator w celu lepszego rozróżniania przykładów _prawdziwych_ (z prior) od _wygenerowanych_ (wyliczonych przez enkoder) [pierwszy krok]
    2. poprawia generator (czyli enkoder auto-enkodera) dla zmylenia dyskryminatora [drugi krok]
* generator może być
  * deterministyczny -- źródłem stochastyczności $q(z)$ pozostaje rozkład danych $p_X(x)$
  * stochastyczny -- $q(z\mid x)\sim\mathcal{N}()$ (jak VAE)

## AAE a inne modele
* VAE
  * bardzo podobny w założeniu
  * w AAE próba dopasowania zagregowanego posterioru $q(z\mid x)$ do założonego prioru $p(z)$
  * VAE korzysta z KL, a tu uczenie adwersarialne
  * składnik KL w kosztcie VAE $$\mathbb{E}_{q(z\mid x)}\log p(x\mid z)-KL(q(z\mid x)\mid\mid p(z))$$ jest zastąpiony składnikiem uczenia adwersarialnego
* GAN
  * GAN (w swojej podstawowej postaci) opiera się na adwersarialnym wymuszeniu rozkładu danych _na poziomie pikseli_ na wyjście modelu
  * AAE uczą autoenkoder dla wykryciu rozkładu danych przez model
  * to powinno pozwalać na wymodelowanie cech _semantycznych_
  * AAE uczy się także na prostszym rozkładzie (gausowski vs. rozkład danych) i w niższej wymiarowości warstwy latent
<img src="../nn_figures/adversarial_autoencoder_trained.pdf" width="90%">

## Regularyzacja przez dodanie etykiet
* dodanie etykiet ułatwi generowanie rozkładów poszczególnych klas
<img src="../nn_figures/adversarial_autoencoder_supervised.pdf" width="70%">
* dodatkowy wektor klasyfikacji (one-hot) dodawany do wejścia dyskryminatora
  * rozdziela granicę decyzyjną na warunkową zależnie od danej etykiety
  * dodatkowa pozycja dla nie etykietowanych przykładów
  * to daje miksturę gausów dla różnych klas
    * w fazie negatywnej uzenia adwersarialnego podawana jest etykieta przykładu uczącego
    * w fazie pozytywnej podawana jest etykieta składnika mikstury z której jest losowany
  
<img src="../nn_figures/adversarial_autoencoder_supervised_trained.pdf" width="90%">  

## AAE semi-nadzorowany
* dane są opisywane w warstwie latent przez
  * rozkład klas $p(y)\sim Cat(y)$ (kategoryczny prior)
  * rozkład opisujacy dane $p(z)\sim\mathcal{N}(0,\mathbb{I})$ (ciągły prior)
<img src="../nn_figures/aae_semisupervised.pdf" width="80%">
* uczenie
  * __rekonstrukcja__
    * model generatora przewiduje $y$ oraz mapuje $x$ do ciągłego $z$ przez enkoder $q(z, y\mid x)$
    *  dekoder wykorzystuje $y, z$ do odtworzenia $x$ z minimalnym błędem
  * __regularyzacja__
    * dyskryminatory starają się rozróżnić 
      * pozytywne przykłady wygenerowane z dyskretnego prior $Cat(y)$ od utworzonego przez enkoder (negatywne)
      * pozytywne z ciągłego prioru $z\sim p(z)$ od negatywnych z enkodera
    * poprawia enkoder dla zmylenia dyskryminatorów
  * __semi-nadzorowanej klasyfikacji__
    * autoenkoder poprawia $q(y\mid x)$ minimalizując entropię krzyżową na _etykietowanym_ mini-batchu
* wyniki uczenia znacznie lepsze od modelu VAE, ale też słabsze od kilku innych